In [75]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostRegressor
from algolab import API
from datetime import datetime
from config import *
import pandas as pd, numpy as np, json, os

import yfinance as yf

In [76]:
son=pd.read_excel("./excel/FiyatPenceresi.Xlsx")
endeks=son.iloc[-1,:]
son=son.iloc[:-1,:]
hisseler=son["Kod"].values
hisseler=np.array(hisseler)
for i in range(len(hisseler)):
    hisseler[i]=hisseler[i]+".IS"

In [77]:
df=pd.DataFrame()
for hisse in hisseler:
    hisse_verileri = yf.download(hisse,start="2010-01-01")  # Örnek tarih aralığı
    hisse_kapanis = hisse_verileri[['High','Low','Open','Close','Volume']]
    hisse_kapanis["Hisse"]=hisse
    hisse_kapanis.loc[hisse_kapanis["Close"] > hisse_kapanis["High"], "High"] = hisse_kapanis["Close"]
    hisse_kapanis.loc[hisse_kapanis["Close"] < hisse_kapanis["Low"], "Low"] = hisse_kapanis["Close"]

    df=pd.concat([df,hisse_kapanis],axis=0)
df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,High,Low,Open,Close,Volume,Hisse
Date,,,,,,
2010-01-01,16.60,16.60,16.600000,16.600000,0,AEFES.IS
2010-01-04,16.90,16.60,16.799999,16.700001,82273,AEFES.IS
2010-01-05,16.90,16.50,16.799999,16.600000,266369,AEFES.IS
2010-01-06,16.90,16.40,16.500000,16.799999,250455,AEFES.IS
2010-01-07,16.90,16.60,16.799999,16.900000,140106,AEFES.IS
...,...,...,...,...,...,...
2024-05-28,6.17,6.02,6.090000,6.020000,154819956,ZOREN.IS
2024-05-29,6.15,5.87,6.040000,5.890000,336346594,ZOREN.IS
2024-05-30,5.94,5.70,5.880000,5.720000,187292178,ZOREN.IS


In [80]:
data=df.copy()

In [81]:
df["Close"]=np.round(df["Close"],2)
df["High"]=np.round(df["High"],2)
df["Low"]=np.round(df["Low"],2)
df["Open"]=np.round(df["Open"],2)
df["Volume"]=df["Volume"]*df["Close"]
df.columns=['High', 'Low', 'Open', 'Adj Close', 'Volume', 'Hisse']

In [82]:
canlı=pd.read_excel("./excel/FiyatPenceresi.Xlsx")
endeks=canlı.iloc[-1,:]
canlı=canlı.iloc[:-1,:]
canlı=canlı[["Yks","Dşk","Açl.Gün","Son.Fyt","Hcm","Kod","Lot"]]
canlı.columns=['High', 'Low', 'Open', 'Adj Close', 'Volume', 'Hisse',"Lot"]
canlı.index = pd.to_datetime(['2024-05-30'] * len(canlı))
canlı = canlı[~canlı['Hisse'].isin(['IDEAS', 'ISATR', 'ISKUR', 'ITTFH'])]
canlı["Lot"]=canlı["Lot"].astype("string")
canlı

,High,Low,Open,Adj Close,Volume,Hisse,Lot
2024-05-30,"214,00","200,50","200,50","214,00",794.380.402,AEFES,3.799.315
2024-05-30,"19,44","18,05","18,80","18,28",1.975.198.660,AGROT,105.340.402
2024-05-30,"14,59","13,95","14,58","13,98",104.354.323,AHGAZ,7.402.837
2024-05-30,"67,40","64,60","66,45","66,80",4.228.810.835,AKBNK,63.835.617
2024-05-30,"138,40","132,20","134,80","136,60",250.382.628,AKCNS,1.861.453
...,...,...,...,...,...,...,...
2024-05-30,"86,00","82,65","84,45","86,00",508.091.197,VESTL,6.029.380
2024-05-30,"194,90","182,40","185,00","188,60",400.966.942,YEOTK,2.110.043
2024-05-30,"33,12","31,68","32,42","32,88",4.935.929.220,YKBNK,151.844.673
2024-05-30,"12,62","12,30","12,62","12,34",99.802.956,YYLGD,8.060.788


In [83]:
for i in range(len(canlı)):
    if len(canlı.iloc[i, -1]) <= 7:
        if '.' in canlı.iloc[i, -1]:
            parts = canlı.iloc[i, -1].split('.')
            decimal_part = parts[1]
            if len(decimal_part) == 1:
                decimal_part += '00'
            elif len(decimal_part) == 2:
                decimal_part += '0'
            elif len(decimal_part) == 3:
                canlı.iloc[i, -1] = parts[0] + decimal_part[:2]
            canlı.iloc[i, -1] = parts[0] + decimal_part

In [84]:
df = df[~df['Hisse'].isin(['ISATR.IS', 'ISKUR.IS'])]

In [85]:
kod=canlı["Hisse"].unique()
for i in range(len(kod)):
    kod[i]=kod[i]+".IS"

In [86]:
canlı["Hisse"]=kod
canlı["High"]=canlı["High"].str.replace(",",".")
canlı["Low"]=canlı["Low"].str.replace(",",".")
canlı["Open"]=canlı["Open"].str.replace(",",".")
canlı["Adj Close"]=canlı["Adj Close"].str.replace(",",".")
canlı["Lot"]=canlı["Lot"].astype(str).str.replace(".", "")
canlı[["High","Low","Open","Adj Close","Lot"]]=canlı[["High","Low","Open","Adj Close","Lot"]].astype(float)
canlı["Volume"]=canlı["Lot"]*canlı["Adj Close"]
canlı

,High,Low,Open,Adj Close,Volume,Hisse,Lot
2024-05-30,214.00,200.50,200.50,214.00,8.130534e+08,AEFES.IS,3799315.0
2024-05-30,19.44,18.05,18.80,18.28,1.925623e+09,AGROT.IS,105340402.0
2024-05-30,14.59,13.95,14.58,13.98,1.034917e+08,AHGAZ.IS,7402837.0
2024-05-30,67.40,64.60,66.45,66.80,4.264219e+09,AKBNK.IS,63835617.0
2024-05-30,138.40,132.20,134.80,136.60,2.542745e+08,AKCNS.IS,1861453.0
...,...,...,...,...,...,...,...
2024-05-30,86.00,82.65,84.45,86.00,5.185267e+08,VESTL.IS,6029380.0
2024-05-30,194.90,182.40,185.00,188.60,3.979541e+08,YEOTK.IS,2110043.0
2024-05-30,33.12,31.68,32.42,32.88,4.992653e+09,YKBNK.IS,151844673.0
2024-05-30,12.62,12.30,12.62,12.34,9.947012e+07,YYLGD.IS,8060788.0


In [87]:
df=df[df["Hisse"].isin(canlı["Hisse"].values)]

In [88]:
df["High"].loc["2024-06-03"]=canlı["High"].values
df["Low"].loc["2024-06-03"]=canlı["Low"].values
df["Open"].loc["2024-06-03"]=canlı["Open"].values
df["Adj Close"].loc["2024-06-03"]=canlı["Adj Close"].values
df["Volume"].loc["2024-06-03"]=canlı["Volume"].values

C:\Users\Bora\AppData\Local\Temp\ipykernel_3392\3524331428.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["High"].loc["2024-06-03"]=canlı["High"].values
C:\Users\Bora\AppData\Local\Temp\ipykernel_3392\3524331428.py:1: SettingWithCopy

In [89]:
df["Kademe"] = np.where(df["Adj Close"] < 20, 0.01,
                np.where(df["Adj Close"] < 50, 0.02,
                    np.where(df["Adj Close"] < 100, 0.05,
                        np.where(df["Adj Close"] < 250, 0.1,
                            np.where(df["Adj Close"] < 500, 0.25,
                                np.where(df["Adj Close"] < 1000, 0.5,
                                    np.where(df["Adj Close"] < 2500, 1, 2.5)
                                )  # Buradaki parantez eksikti.
                            )
                        )
                    )
                )
            )

In [90]:
import math
def yuvarla_asagi(row):
    fiyat = row['Adj Close']
    kadem = row['Kademe']
    return math.floor(fiyat * 1.1 / kadem) * kadem

df["Tavan"]=df.apply(yuvarla_asagi, axis=1)
df["Tavan"]=np.round(df["Tavan"],2)

In [91]:
df["Tavan Kontrol"]=np.where(((df["Adj Close"]/df["Tavan"].shift(1))==1),1,0)
df=df.drop(["Kademe","Tavan"],axis=1)

In [92]:
import pandas_ta as ta

df["SMA 5"]=df["Adj Close"].rolling(5).mean()
df["SMA 10"]=df["Adj Close"].rolling(5).mean()
df["SMA 200"]=df["Adj Close"].rolling(5).mean()

# RSI
df["RSI"] = ta.rsi(df["Adj Close"],14)

# Bağıl Hacim
df["Bağıl Hacim"] = df["Volume"] / df["Volume"].rolling(10).mean()



# ADX, DMI, Aroon
adx_data = ta.adx(df["High"], df["Low"], df["Adj Close"], 14)
df["ADX"], df["DMIP"], df["DMIN"] = adx_data.iloc[:, 0], adx_data.iloc[:, 1], adx_data.iloc[:, 2]




def calculate_stoch_rsi(data, rsi_period=14, stochastic_period=14, k_period=3, d_period=3):
    rsi = ta.rsi(data["Adj Close"], rsi_period)
    stoch_rsi = (rsi - rsi.rolling(window=stochastic_period).min()) / (rsi.rolling(window=stochastic_period).max() - rsi.rolling(window=stochastic_period).min())
    stoch_rsi_k = stoch_rsi.rolling(window=k_period).mean() * 100
    stoch_rsi_d = stoch_rsi_k.rolling(window=d_period).mean()
    data['StochRSI_%K'] = stoch_rsi_k
    data['StochRSI_%D'] = stoch_rsi_d
    return data

df = calculate_stoch_rsi(df)


def calculate_macd(data, short_period=12, long_period=26, signal_period=9):
    data['EMA_12'] = ta.ema(data['Adj Close'], short_period)
    data['EMA_26'] = ta.ema(data['Adj Close'], long_period)
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    data['Signal_Line'] = ta.ema(data['MACD'], signal_period)
    data['MACD_above_Signal'] = (data['MACD'] > data['Signal_Line']).astype(float)
    return data

df = calculate_macd(df)



# Diğer Göstergeler
df["DD"] = ((df["Adj Close"] / df["High"]) - 1) * 100
df["Range"] = ((df["High"] - df["Low"]) / df["Adj Close"]) * 100

df["MOM"] = ta.mom(df["Adj Close"], 10)



df["Driehaus Momentum"] = ((df["Bağıl Hacim"] > 2) & (df["RSI"] > 50) & (df["MOM"] > 1) & (df["Adj Close"] > df["SMA 5"]) & (df["Adj Close"] > df["SMA 10"]) & (df["Adj Close"] > df["SMA 200"])).astype(int)
df["MACD Yukarı Kesen"] = ((df["Bağıl Hacim"] > 1.5) & (df["MACD"] > 0) & (df["MACD_above_Signal"] == 1)).astype(int)
df["ADXDMI20"] = ((df["Bağıl Hacim"] > 1.3) & (df["ADX"] > 20) & (df["DMIP"] > df["DMIN"])).astype(int)
df["MACD+Stoch"] = ((df["Bağıl Hacim"] > 1.3) & (df["StochRSI_%K"] > df["StochRSI_%D"]) & (df["MACD_above_Signal"] == 1)).astype(int)




In [93]:
df["Return"]=100*((df["Open"]/df["Adj Close"].shift(2))-1)
df["Return"]=df["Return"].shift(-2)
df["Gerçek Return"]=100*((df["Open"]/df["Open"].shift(1))-1)
df["Gerçek Return"]=df["Gerçek Return"].shift(-2)
df["Date"]=pd.to_datetime(df.index)

In [94]:
df=df[['High', 'Low', 'Open', 'Adj Close', 'Volume', 'Hisse', 'Bağıl Hacim', 'RSI','Driehaus Momentum', 'MACD Yukarı Kesen', 'ADXDMI20',
       'MACD+Stoch','DD','Range','Return',"Gerçek Return","Tavan Kontrol"]]

In [95]:
endeks=endeks["Son.Fyt"]
endeks=endeks.replace(",",".")
endeks=float(endeks)

In [96]:
index=yf.download(tickers="XU100.IS",start="2000-01-01")
df["Endeks"]=index["Adj Close"]
df["Endeks Return"]=100*((df["Endeks"]/df["Endeks"].shift(1))-1)
df["Return Lag"]=((df["Adj Close"]/df["Adj Close"].shift(1))-1)*100
endeks_son=((endeks/index["Adj Close"].iloc[-2])-1)*100
df["Endeks Return"].loc["2024-06-03"]=endeks_son
df

[*********************100%%**********************]  1 of 1 completed
C:\Users\Bora\AppData\Local\Temp\ipykernel_3392\2776133884.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Endeks Return"].loc["2024-06-03"]=endeks_son
C:\Users\Bor

,High,Low,Open,Adj Close,Volume,Hisse,Bağıl Hacim,RSI,Driehaus Momentum,MACD Yukarı Kesen,ADXDMI20,MACD+Stoch,DD,Range,Return,Gerçek Return,Tavan Kontrol,Endeks,Endeks Return,Return Lag
Date,,,,,,,,,,,,,,,,,,,,
2010-01-01,16.60,16.60,16.60,16.60,0.000000e+00,AEFES.IS,NaN,NaN,0,0,0,0,0.000000,0.000000,1.204819,0.000000,0,NaN,NaN,NaN
2010-01-04,16.90,16.60,16.80,16.70,1.373959e+06,AEFES.IS,NaN,NaN,0,0,0,0,-1.183432,1.796407,-1.197605,-1.785714,0,53368.199219,NaN,0.602410
2010-01-05,16.90,16.50,16.80,16.60,4.421725e+06,AEFES.IS,NaN,NaN,0,0,0,0,-1.775148,2.409639,1.204819,1.818182,0,54115.101562,1.399527,-0.598802
2010-01-06,16.90,16.40,16.50,16.80,4.207644e+06,AEFES.IS,NaN,NaN,0,0,0,0,-0.591716,2.976190,1.190476,1.190476,0,54547.398438,0.798847,1.204819
2010-01-07,16.90,16.60,16.80,16.90,2.367791e+06,AEFES.IS,NaN,NaN,0,0,0,0,0.000000,1.775148,0.591716,0.000000,0,54972.898438,0.780056,0.595238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-28,6.17,6.02,6.09,6.02,9.320161e+08,ZOREN.IS,0.698944,46.981841,0,0,0,0,-2.431118,2.491694,-2.325581,-2.649007,0,10677.500000,0.192362,-1.149425
2024-05-29,6.15,5.87,6.04,5.89,1.981081e+09,ZOREN.IS,1.368453,43.468572,0,0,0,0,-4.227642,4.753820,-2.376910,-2.210884,0,10479.200195,-1.857174,-2.159468
2024-05-30,5.94,5.70,5.88,5.72,1.071311e+09,ZOREN.IS,0.799376,39.327024,0,0,0,0,-3.703704,4.195804,-1.048951,-1.565217,0,10481.299805,0.020036,-2.886248


In [97]:
df=df.dropna()
test=df.loc["2024-06-03"]
X_test=test.drop(["High","Low","Open","Adj Close","Hisse","Return","Endeks","Tavan Kontrol","Gerçek Return"],axis=1)

In [98]:
params={'iterations': 571,
 'depth': 5,
 'learning_rate': 0.05514261151828995,
 'random_strength': 4,
 'bagging_temperature': 0.9572252022114495,
 'border_count': 59,
 'l2_leaf_reg': 20}
model=CatBoostRegressor(**params,task_type="GPU",random_seed=123)
model.load_model("./model/bist100model.bin")

In [99]:
tahminler=pd.DataFrame()
tahminler["Hisse"]=test["Hisse"].values
tahminler["Tavan"]=test["Tavan Kontrol"].values
tahminler["Tahmin"]=model.predict(X_test)
tahminler=tahminler[tahminler["Tavan"]==0]
tahminler=tahminler.sort_values(by="Tahmin",ascending=False).head(5)
tahminler

,Hisse,Tavan,Tahmin
42,GARAN.IS,0,1.239276
91,TURSG.IS,0,1.198218
82,TCELL.IS,0,1.068330
61,KRDMD.IS,0,1.010826
92,ULKER.IS,0,0.983401


In [100]:
indexler=df.index.unique()
tahminler=tahminler.set_index(pd.to_datetime(["2024-06-03","2024-06-03","2024-06-03","2024-06-03","2024-06-03"]))

In [101]:
window_size=10
vars=[]
tahminler["Var"]=0
for index, group in tahminler.groupby(level=0):
    hisseler=group["Hisse"]
    for hisse in hisseler:
        try:
           window_data=df[df["Hisse"]==hisse]["Adj Close"].loc[indexler[-500]:].loc[:index].iloc[-window_size:]
           getiriler=window_data.pct_change(window_size-1)*100
           getiri=getiriler.iloc[-1]
           std=window_data.std()
           var=getiri/std
           vars.append(var)
           tahminler.loc[(tahminler["Hisse"]==hisse) & (tahminler.index==index), "Var"] = var
        except Exception as e:
            print(e)
tahminler

C:\Users\Bora\AppData\Local\Temp\ipykernel_3392\1541433454.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.0498000516334245' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tahminler.loc[(tahminler["Hisse"]==hisse) & (tahminler.index==index), "Var"] = var


,Hisse,Tavan,Tahmin,Var
2024-06-03,GARAN.IS,0,1.239276,2.049800
2024-06-03,TURSG.IS,0,1.198218,3.026045
2024-06-03,TCELL.IS,0,1.068330,3.094086
2024-06-03,KRDMD.IS,0,1.010826,8.407792
2024-06-03,ULKER.IS,0,0.983401,1.829944


In [102]:
bakiye=4051.02
veri=tahminler[tahminler["Var"]>0]
veri=veri.head(2)
veri['Ağırlık'] = veri['Var'] / veri.groupby(level=0)['Var'].transform('sum')
veri["Fiyat"]=test[test["Hisse"].isin(veri["Hisse"])]["Adj Close"].values
veri["Para"]=bakiye*veri["Ağırlık"]
veri["Lot"]=veri["Para"]//(veri["Fiyat"]*1.1)
veri

,Hisse,Tavan,Tahmin,Var,Ağırlık,Fiyat,Para,Lot
2024-06-03,GARAN.IS,0,1.239276,2.049800,0.403834,108.6,1635.940542,13.0
2024-06-03,TURSG.IS,0,1.198218,3.026045,0.596166,80.5,2415.079458,27.0
